In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/My Drive/수업/Info_Sys_2021/PyTorch

/content/drive/My Drive/수업/Info_Sys_2021/PyTorch


In [ ]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

import torch.nn as nn

from torchsummary import summary


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class MultiLayerModel_3(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(nn.Linear(28*28, 256), nn.ReLU(), nn.Dropout(p=drop_prob)) 
        self.layer2 = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Dropout(p=drop_prob))
        self.layer3 = nn.Sequential(nn.Linear(256, 64), nn.ReLU(), nn.Dropout(p=drop_prob))
        self.layer4 = nn.Linear(64, 10)

    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
               
        return out

In [ ]:
model = MultiLayerModel_3().to(device)

In [ ]:
model

MultiLayerModel_3(
  (layer1): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (layer4): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
print(summary(model,(1,28*28)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 256]         200,960
              ReLU-2               [-1, 1, 256]               0
           Dropout-3               [-1, 1, 256]               0
            Linear-4               [-1, 1, 256]          65,792
              ReLU-5               [-1, 1, 256]               0
           Dropout-6               [-1, 1, 256]               0
            Linear-7                [-1, 1, 64]          16,448
              ReLU-8                [-1, 1, 64]               0
           Dropout-9                [-1, 1, 64]               0
           Linear-10                [-1, 1, 10]             650
Total params: 283,850
Trainable params: 283,850
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.08
Estimated T

In [ ]:
pip install pytorch_model_summary

In [ ]:
import pytorch_model_summary

In [ ]:
print(pytorch_model_summary.summary(model, torch.zeros(1, 28*28).to(device), show_input=False))

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Linear-1            [1, 256]         200,960         200,960
            ReLU-2            [1, 256]               0               0
         Dropout-3            [1, 256]               0               0
          Linear-4            [1, 256]          65,792          65,792
            ReLU-5            [1, 256]               0               0
         Dropout-6            [1, 256]               0               0
          Linear-7             [1, 64]          16,448          16,448
            ReLU-8             [1, 64]               0               0
         Dropout-9             [1, 64]               0               0
         Linear-10             [1, 10]             650             650
Total params: 283,850
Trainable params: 283,850
Non-trainable params: 0
-----------------------------------------------------------------------


In [ ]:
print(pytorch_model_summary.summary(model, torch.zeros(1, 28*28).to(device), show_input=True))

-----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
          Linear-1            [1, 784]         200,960         200,960
            ReLU-2            [1, 256]               0               0
         Dropout-3            [1, 256]               0               0
          Linear-4            [1, 256]          65,792          65,792
            ReLU-5            [1, 256]               0               0
         Dropout-6            [1, 256]               0               0
          Linear-7            [1, 256]          16,448          16,448
            ReLU-8             [1, 64]               0               0
         Dropout-9             [1, 64]               0               0
         Linear-10             [1, 64]             650             650
Total params: 283,850
Trainable params: 283,850
Non-trainable params: 0
-----------------------------------------------------------------------


In [ ]:
for m in model.modules():
    if isinstance(m, (nn.Linear)):
        nn.init.xavier_uniform_(m.weight)

In [ ]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.410202533
Epoch: 0002 cost = 0.176356763
Epoch: 0003 cost = 0.134498402
Epoch: 0004 cost = 0.113907859
Epoch: 0005 cost = 0.099860735
Epoch: 0006 cost = 0.084474914
Epoch: 0007 cost = 0.078666501
Epoch: 0008 cost = 0.072091013
Epoch: 0009 cost = 0.068223543
Epoch: 0010 cost = 0.061535269
Epoch: 0011 cost = 0.057945587
Epoch: 0012 cost = 0.055458017
Epoch: 0013 cost = 0.053393073
Epoch: 0014 cost = 0.051278770
Epoch: 0015 cost = 0.047412928
Learning finished


In [ ]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9800999760627747
Label:  8
Prediction:  8


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [ ]:
X.shape

torch.Size([100, 784])